# (a) First, consider the TechFit Smartwatch with the price response functions provided in Table 2 for the first two weeks (assume no cross-elasticity). Using the KKT conditions, derive the optimal prices, assuming they are non-negative but otherwise have no restrictions.

In [19]:
import pandas as pd
from gurobipy import Model, GRB, LinExpr

# Load the data from the provided CSV file
df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# Extract data for Week 1 and Week 2 for TechFit Smartwatch
week1_intercept = 1000
week1_coeff = -5
week2_intercept = 950
week2_coeff = -4.5

# Create the optimization model
model = Model("Optimal_Pricing")

# Add decision variables for prices in Week 1 and Week 2
P1 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_Week1")
P2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_Week2")

# Define the revenue functions for both weeks
revenue_week1 = P1 * (week1_intercept + week1_coeff * P1)
revenue_week2 = P2 * (week2_intercept + week2_coeff * P2)

# Set the objective to maximize total revenue
model.setObjective(revenue_week1 + revenue_week2, GRB.MAXIMIZE)

# Optimize the model
model.optimize()

# Check and print the results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Price for Week 1: ${P1.X:.2f}")
    print(f"Optimal Price for Week 2: ${P2.X:.2f}")
    print(f"Maximum Total Revenue: ${model.objVal:.2f}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x01fefae1
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+03, 1e+03]
  QObjective range [9e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective 1.00138889e+05
Optimal Price for Week 1: $100.00
Optimal Price for Week 2: $105.56
Maximum Total Revenue: $100138.89


# (b) Consider again the TechFit Smartwatch with the same price response functions as in the previous question for the first two weeks (assume no cross-elasticity). Using the KKT conditions, derive the optimal prices assuming they are non-negative but must remain the same across both weeks.

In [20]:
import pandas as pd
from gurobipy import Model, GRB

# Load the data from the provided CSV file
df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# Extract data for Week 1 and Week 2 for TechFit Smartwatch
week1_intercept = 1000
week1_coeff = -5
week2_intercept = 950
week2_coeff = -4.5

# Create the optimization model
model = Model("Optimal_Same_Price")

# Add a decision variable for the common price in both weeks
P = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Common_Price")

# Define the total revenue function with the same price across both weeks
total_revenue = P * (week1_intercept + week1_coeff * P) + P * (week2_intercept + week2_coeff * P)

# Set the objective to maximize total revenue
model.setObjective(total_revenue, GRB.MAXIMIZE)

# Optimize the model
model.optimize()

# Check and print the results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Common Price for Week 1 and Week 2: ${P.X:.2f}")
    print(f"Maximum Total Revenue: ${model.objVal:.2f}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 1 columns and 0 nonzeros
Model fingerprint: 0x36313fc2
Model has 1 quadratic objective term
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [2e+03, 2e+03]
  QObjective range [2e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.04 seconds (0.00 work units)
Optimal objective 1.00065789e+05
Optimal Common Price for Week 1 and Week 2: $102.63
Maximum Total Revenue: $100065.79


# (c) What do you observe about the optimal prices derived using the KKT conditions with and without the equality constraint? Based on our class discussion from the Variable Pricing with Diversion example, why does this occur? What is the benefit of dynamic pricing?

### **(c) Answer – Observations and Analysis**

We can observe the following differences between the results in **(a)** and **(b)**:

---

### **1. Observations on Optimal Prices and Revenue**

1. **Optimal Prices (Without Equality Constraint):**
   - **Week 1 Optimal Price (\( P_1 \)):** $100.00  
   - **Week 2 Optimal Price (\( P_2 \)):** $105.56  
   - **Maximum Total Revenue:** $100,138.89  

2. **Optimal Price (With Equality Constraint):**
   - **Common Price (\( P_1 = P_2 \)):** $102.63  
   - **Maximum Total Revenue:** $100,065.79  

3. **Revenue Difference:**  
   The maximum total revenue without the equality constraint is **$100,138.89**, which is **$73.10 higher** than the revenue with the equality constraint ($100,065.79). This is because dynamic pricing allows each week to have a different optimal price, better aligning with the demand-response functions for those weeks.

---

### **2. Explanation – Why This Occurs**
- **Variable Pricing Advantage:**  
  In (a), we allowed the prices for Week 1 and Week 2 to be optimized independently. This flexibility enables the price to better match the demand curve for each week, resulting in higher revenue.  
  In (b), the same price is enforced across both weeks, which represents a trade-off between the two demand curves, leading to a slightly suboptimal outcome in both weeks.  

- **Demand Curves Matter:**  
  - In Week 1, the optimal price was $100, which aligned perfectly with the demand curve (Intercept: 1000, Own-Price Coefficient: -5).  
  - In Week 2, the demand curve had a different structure (Intercept: 950, Own-Price Coefficient: -4.5), leading to a different optimal price of $105.56 when optimized independently.

- **KKT Conditions and Flexibility:**  
  - Without the equality constraint, the KKT conditions found the true optimal solution for each week.  
  - Adding the constraint \( P_1 = P_2 \) introduces a compromise, where the price ($102.63) falls between the two optimal prices from (a).

---

### **3. Benefit of Dynamic Pricing**
- **Higher Revenue Potential:**  
  As shown by the difference in total revenue, dynamic pricing allows the company to extract more value by adapting prices to weekly demand patterns. This flexibility is critical in maximizing revenue.  
- **Better Demand Matching:**  
  By adjusting prices weekly, the company can better capture the willingness to pay of different consumer segments at different times.  
- **Seasonality and Market Responsiveness:**  
  Dynamic pricing enables the company to respond to seasonality and promotional events (e.g., Black Friday), further enhancing profitability.  

---

### **Conclusion**
- **Static Pricing** simplifies operations but sacrifices potential revenue.  
- **Dynamic Pricing** offers superior performance by allowing prices to vary in response to demand changes, leading to optimal results across different market conditions.

---

If you'd like, I can also help visualize these differences using a plot comparing the price and revenue under the two scenarios. 😊

# (d) Now consider both products. Using the price response functions in the price response.csv file for the first two weeks only, determine the optimal prices using the projected gradient descent algorithm. For each product, assume static pricing across both weeks. Initialize all prices at zero, with a step size of 0.001 and a stopping criterion of 10−6. What are the optimal prices?

In [21]:
# import pandas as pd
# import numpy as np

# # Load the data
# df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# # Filter data for the first two weeks and both products
# techfit_weeks = df[(df['Product'] == 'TechFit Smartwatch') & (df['Week'] <= 2)]
# powersound_weeks = df[(df['Product'] == 'PowerSound Earbuds') & (df['Week'] <= 2)]

# # Extract coefficients
# intercept_techfit = techfit_weeks['Intercept'].mean()
# coeff_techfit = techfit_weeks['Own_Price_Coefficient'].mean()

# intercept_powersound = powersound_weeks['Intercept'].mean()
# coeff_powersound = powersound_weeks['Own_Price_Coefficient'].mean()

# # Initialization
# P_techfit = 0.0
# P_powersound = 0.0
# step_size = 0.001
# tolerance = 1e-6
# max_iter = 100000

# # Projected Gradient Descent
# for i in range(max_iter):
#     # Compute gradients
#     grad_techfit = intercept_techfit + 2 * coeff_techfit * P_techfit
#     grad_powersound = intercept_powersound + 2 * coeff_powersound * P_powersound
    
#     # Update prices
#     new_P_techfit = P_techfit + step_size * grad_techfit
#     new_P_powersound = P_powersound + step_size * grad_powersound
    
#     # Projection to ensure non-negative prices
#     new_P_techfit = max(0, new_P_techfit)
#     new_P_powersound = max(0, new_P_powersound)
    
#     # Check stopping criterion
#     if np.linalg.norm([new_P_techfit - P_techfit, new_P_powersound - P_powersound]) < tolerance:
#         break
    
#     # Update current prices
#     P_techfit, P_powersound = new_P_techfit, new_P_powersound

# # Print the optimal prices
# print(f"Optimal Price for TechFit Smartwatch: ${P_techfit:.2f}")
# print(f"Optimal Price for PowerSound Earbuds: ${P_powersound:.2f}")


Optimal Price for TechFit Smartwatch: $82.44
Optimal Price for PowerSound Earbuds: $98.34


In [22]:
# from gurobipy import Model, GRB

# # Create the optimization model
# model = Model("Optimal_Pricing_PGD_Verification")

# # Add decision variables for prices
# P_techfit = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_TechFit")
# P_powersound = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_PowerSound")

# # Define the revenue functions for both products
# revenue_techfit = P_techfit * (intercept_techfit + coeff_techfit * P_techfit)
# revenue_powersound = P_powersound * (intercept_powersound + coeff_powersound * P_powersound)

# # Set the objective to maximize total revenue
# model.setObjective(revenue_techfit + revenue_powersound, GRB.MAXIMIZE)

# # Optimize the model
# model.optimize()

# # Print the optimal prices and maximum revenue
# if model.status == GRB.OPTIMAL:
#     print(f"Gurobi - Optimal Price for TechFit Smartwatch: ${P_techfit.X:.2f}")
#     print(f"Gurobi - Optimal Price for PowerSound Earbuds: ${P_powersound.X:.2f}")
#     print(f"Gurobi - Maximum Total Revenue: ${model.objVal:.2f}")
# else:
#     print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x8514b80a
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [3e+02, 3e+02]
  QObjective range [3e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.03 seconds (0.00 work units)
Optimal objective 2.44635110e+04
Gurobi - Optimal Price for TechFit Smartwatch: $82.44
Gurobi - Optimal Price for PowerSound Earbuds: $98.34
Gurobi - Maximum Total Revenue: $24463.51


In [31]:
import pandas as pd
import numpy as np

# Load the data from the provided CSV file
df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# Filter data for Week 1 and Week 2
data_week1 = df[df['Week'] == 1]
data_week2 = df[df['Week'] == 2]

# Initialize prices for both products at zero
price_smartwatch = 0.0
price_earbuds = 0.0

# Set step size and stopping criterion
step_size = 0.001
tolerance = 1e-6
max_iterations = 10000

def calculate_demand(intercept, own_coeff, cross_coeff, own_price, cross_price):
    """ Calculate demand based on the given coefficients and prices. """
    return intercept + own_coeff * own_price + cross_coeff * cross_price

def calculate_revenue(intercept, own_coeff, cross_coeff, own_price, cross_price):
    """ Calculate revenue as price times demand. """
    demand = calculate_demand(intercept, own_coeff, cross_coeff, own_price, cross_price)
    return own_price * demand

def gradient_descent(prices, intercepts, own_coeffs, cross_coeffs, step_size, tolerance, max_iterations):
    """ Perform Projected Gradient Descent to optimize prices. """
    prev_prices = np.array(prices)
    for iteration in range(max_iterations):
        gradients = []
        for i in range(2):
            own_price = prev_prices[i]
            cross_price = prev_prices[1 - i]
            # Calculate partial derivative of revenue with respect to the own price
            demand = calculate_demand(intercepts[i], own_coeffs[i], cross_coeffs[i], own_price, cross_price)
            gradient = demand + own_price * own_coeffs[i]
            gradients.append(gradient)
        
        # Update prices using the gradients
        new_prices = prev_prices + step_size * np.array(gradients)
        
        # Project prices to be non-negative
        new_prices = np.maximum(new_prices, 0)
        
        # Check for convergence
        if np.linalg.norm(new_prices - prev_prices) < tolerance:
            print(f"Converged in {iteration} iterations.")
            break
        prev_prices = new_prices
    
    return new_prices

# Coefficients and intercepts for TechFit Smartwatch and PowerSound Earbuds for Week 1 and Week 2
intercepts = [data_week1['Intercept'].values[0], data_week1['Intercept'].values[1]]
own_coeffs = [data_week1['Own_Price_Coefficient'].values[0], data_week1['Own_Price_Coefficient'].values[1]]
cross_coeffs = [data_week1['Cross_Price_Coefficient'].values[0], data_week1['Cross_Price_Coefficient'].values[1]]

# Perform gradient descent
optimal_prices = gradient_descent(
    prices=[price_smartwatch, price_earbuds],
    intercepts=intercepts,
    own_coeffs=own_coeffs,
    cross_coeffs=cross_coeffs,
    step_size=step_size,
    tolerance=tolerance,
    max_iterations=max_iterations
)

print(f"Optimal Price for TechFit Smartwatch: ${optimal_prices[0]:.2f}")
print(f"Optimal Price for PowerSound Earbuds: ${optimal_prices[1]:.2f}")


Converged in 4605 iterations.
Optimal Price for TechFit Smartwatch: $107.63
Optimal Price for PowerSound Earbuds: $94.93


In [32]:
import pandas as pd
from gurobipy import Model, GRB, QuadExpr

# Load the data from the provided CSV file
df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# Filter data for Week 1 and Week 2
data_week1 = df[df['Week'] == 1]
data_week2 = df[df['Week'] == 2]

# Extract coefficients and intercepts for both products
intercept_smartwatch = data_week1[data_week1['Product'] == 'TechFit Smartwatch']['Intercept'].values[0]
own_coeff_smartwatch = data_week1[data_week1['Product'] == 'TechFit Smartwatch']['Own_Price_Coefficient'].values[0]
cross_coeff_smartwatch = data_week1[data_week1['Product'] == 'TechFit Smartwatch']['Cross_Price_Coefficient'].values[0]

intercept_earbuds = data_week1[data_week1['Product'] == 'PowerSound Earbuds']['Intercept'].values[0]
own_coeff_earbuds = data_week1[data_week1['Product'] == 'PowerSound Earbuds']['Own_Price_Coefficient'].values[0]
cross_coeff_earbuds = data_week1[data_week1['Product'] == 'PowerSound Earbuds']['Cross_Price_Coefficient'].values[0]

# Create the optimization model
model = Model("Optimal_Pricing_with_Cross_Elasticity")

# Add decision variables for the prices of both products
P_smartwatch = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_TechFit_Smartwatch")
P_earbuds = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_PowerSound_Earbuds")

# Define the revenue functions for both products
revenue_smartwatch = P_smartwatch * (intercept_smartwatch + own_coeff_smartwatch * P_smartwatch + cross_coeff_smartwatch * P_earbuds)
revenue_earbuds = P_earbuds * (intercept_earbuds + own_coeff_earbuds * P_earbuds + cross_coeff_earbuds * P_smartwatch)

# Set the objective to maximize total revenue (combined for both products)
model.setObjective(revenue_smartwatch + revenue_earbuds, GRB.MAXIMIZE)

# Optimize the model
model.optimize()

# Check and print the results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Price for TechFit Smartwatch: ${P_smartwatch.X:.2f}")
    print(f"Optimal Price for PowerSound Earbuds: ${P_earbuds.X:.2f}")
    print(f"Maximum Total Revenue: ${model.objVal:.2f}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x54120bf9
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [3e+02, 3e+02]
  QObjective range [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 0 rows, 2 columns, 0 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 1
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -5.03006734e+06  6.16825626e+06  8.21e+02 1.84e+01  9.82e+05     0s
   1  -3.

In [28]:
from gurobipy import Model, GRB
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# Extract coefficients for the first two weeks for both products
coeffs = df.set_index(['Week', 'Product'])[['Intercept', 'Own_Price_Coefficient', 'Cross_Price_Coefficient']]

# TechFit Smartwatch coefficients for Weeks 1 & 2
alpha_s_1 = coeffs.loc[(1, 'TechFit Smartwatch'), 'Intercept']
beta_s_1 = coeffs.loc[(1, 'TechFit Smartwatch'), 'Own_Price_Coefficient']
gamma_s_1 = coeffs.loc[(1, 'TechFit Smartwatch'), 'Cross_Price_Coefficient']

alpha_s_2 = coeffs.loc[(2, 'TechFit Smartwatch'), 'Intercept']
beta_s_2 = coeffs.loc[(2, 'TechFit Smartwatch'), 'Own_Price_Coefficient']
gamma_s_2 = coeffs.loc[(2, 'TechFit Smartwatch'), 'Cross_Price_Coefficient']

# PowerSound Earbuds coefficients for Weeks 1 & 2
alpha_e_1 = coeffs.loc[(1, 'PowerSound Earbuds'), 'Intercept']
beta_e_1 = coeffs.loc[(1, 'PowerSound Earbuds'), 'Own_Price_Coefficient']
gamma_e_1 = coeffs.loc[(1, 'PowerSound Earbuds'), 'Cross_Price_Coefficient']

alpha_e_2 = coeffs.loc[(2, 'PowerSound Earbuds'), 'Intercept']
beta_e_2 = coeffs.loc[(2, 'PowerSound Earbuds'), 'Own_Price_Coefficient']
gamma_e_2 = coeffs.loc[(2, 'PowerSound Earbuds'), 'Cross_Price_Coefficient']

# Initialize prices
P_s, P_e = 0.0, 0.0  # Initial prices for TechFit and PowerSound
eta = 0.001  # Initial step size
tolerance = 1e-6  # Stopping criterion
max_iterations = 100000  # Maximum number of iterations

# Projected Gradient Descent with dynamic step size
for i in range(max_iterations):
    # Compute total demand gradients for both products
    grad_P_s = (
        (alpha_s_1 + beta_s_1 * P_s + gamma_s_1 * P_e) + 
        (alpha_s_2 + beta_s_2 * P_s + gamma_s_2 * P_e)
    )
    grad_P_e = (
        (alpha_e_1 + beta_e_1 * P_e + gamma_e_1 * P_s) + 
        (alpha_e_2 + beta_e_2 * P_e + gamma_e_2 * P_s)
    )

    # Update prices using projected gradient step with dynamic step size
    new_P_s = max(0, P_s + eta * grad_P_s)
    new_P_e = max(0, P_e + eta * grad_P_e)

    # Reduce step size if the update is too large (to prevent skipping the optimal point)
    if abs(new_P_s - P_s) > 10 or abs(new_P_e - P_e) > 10:
        eta *= 0.5

    # Check for convergence
    if abs(new_P_s - P_s) < tolerance and abs(new_P_e - P_e) < tolerance:
        break

    # Update prices
    P_s, P_e = new_P_s, new_P_e

# Print the optimized prices
print(f"Optimal Price for TechFit Smartwatch: ${P_s:.2f}")
print(f"Optimal Price for PowerSound Earbuds: ${P_e:.2f}")


Optimal Price for TechFit Smartwatch: $195.02
Optimal Price for PowerSound Earbuds: $234.27


In [24]:
from gurobipy import Model, GRB
import pandas as pd

# Load the data
df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# Extract coefficients for the first two weeks for both products
coeffs = df.set_index(['Week', 'Product'])[['Intercept', 'Own_Price_Coefficient', 'Cross_Price_Coefficient']]

# TechFit Smartwatch coefficients for Weeks 1 & 2
alpha_s_1 = coeffs.loc[(1, 'TechFit Smartwatch'), 'Intercept']
beta_s_1 = coeffs.loc[(1, 'TechFit Smartwatch'), 'Own_Price_Coefficient']
gamma_s_1 = coeffs.loc[(1, 'TechFit Smartwatch'), 'Cross_Price_Coefficient']

alpha_s_2 = coeffs.loc[(2, 'TechFit Smartwatch'), 'Intercept']
beta_s_2 = coeffs.loc[(2, 'TechFit Smartwatch'), 'Own_Price_Coefficient']
gamma_s_2 = coeffs.loc[(2, 'TechFit Smartwatch'), 'Cross_Price_Coefficient']

# PowerSound Earbuds coefficients for Weeks 1 & 2
alpha_e_1 = coeffs.loc[(1, 'PowerSound Earbuds'), 'Intercept']
beta_e_1 = coeffs.loc[(1, 'PowerSound Earbuds'), 'Own_Price_Coefficient']
gamma_e_1 = coeffs.loc[(1, 'PowerSound Earbuds'), 'Cross_Price_Coefficient']

alpha_e_2 = coeffs.loc[(2, 'PowerSound Earbuds'), 'Intercept']
beta_e_2 = coeffs.loc[(2, 'PowerSound Earbuds'), 'Own_Price_Coefficient']
gamma_e_2 = coeffs.loc[(2, 'PowerSound Earbuds'), 'Cross_Price_Coefficient']

# Create the optimization model
model = Model("Optimal_Pricing_Validation")

# Add decision variables for the prices
P_s = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_TechFit")
P_e = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_PowerSound")

# Define the total revenue function with cross-price effects
revenue_techfit = (
    P_s * (alpha_s_1 + beta_s_1 * P_s + gamma_s_1 * P_e) + 
    P_s * (alpha_s_2 + beta_s_2 * P_s + gamma_s_2 * P_e)
)
revenue_powersound = (
    P_e * (alpha_e_1 + beta_e_1 * P_e + gamma_e_1 * P_s) + 
    P_e * (alpha_e_2 + beta_e_2 * P_e + gamma_e_2 * P_s)
)

# Set the objective to maximize total revenue
model.setObjective(revenue_techfit + revenue_powersound, GRB.MAXIMIZE)

# Optimize the model
model.optimize()

# Check and print the results
if model.status == GRB.OPTIMAL:
    print(f"Gurobi - Optimal Price for TechFit Smartwatch: ${P_s.X:.2f}")
    print(f"Gurobi - Optimal Price for PowerSound Earbuds: ${P_e.X:.2f}")
    print(f"Gurobi - Maximum Total Revenue: ${model.objVal:.2f}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x93c64414
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [5e+02, 6e+02]
  QObjective range [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 0 rows, 2 columns, 0 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 1
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -8.20193478e+06  1.03741354e+07  8.61e+02 1.76e+01  9.83e+05     0s
   1  -6.

# (e) For the full model, why is this optimization problem considered to be a nonlinear program? Discuss why no linear reformulations of the problem are possible.

# (f) Implement and solve the full model across all 17 weeks using Gurobi. Assume that dynamic pricing is allowed. What is the optimal revenue over the 17-week period?

# (g) Generate a plot showing the price dynamics for each product over the 17-week period on the same graph. Briefly comment on whether you think GadgetMarket Inc. and its customers would find these price trends favorable. As a customer, is there anything you would find concerning?

# (h) Benchmarking is useful for contextualization. Using Gurobi, what would the optimal revenue be for a dynamic pricing strategy without any price constraints except to ensure prices and demand are non-negative? Alternatively, what would the optimal revenue be if prices were constrained to be the same for all 17 weeks?

# (i) Compare the benefits and drawbacks of (i) the static pricing model, (ii) the unconstrained dynamic pricing model, and (iii) the constrained dynamic pricing model.